# TD Docker

In [ ]:
!docker -v

# I. Déployer l'application Dockercoins

https://github.com/jpetazzo/container.training/tree/main/dockercoins

Il s'agit d'une application fictive réalisée par https://github.com/jpetazzo

L'objectif de l'application est de simuler du minage de cryptomonnaie à l'aide de 5 services :
- **rng** = service web générant des bytes aléatoires
- **hasher** = service web calculant le hachage des données
- **worker** = processus d'arrière-plan appelant rng et hasher
- **webui** = interface web pour surveiller la progression
- **redis** = data store (contient un compteur mis à jour par worker)

**En pratique :**
Dans une boucle infinie, le worker appelle le service web rng pour générer des bytes aléatoires et appelle le service web `hasher` pour hacher ces octets.
Toutes les secondes, le worker met à jour redis pour indiquer combien de boucles ont été effectuées. La webui interroge redis, calcule et affiche la "vitesse de hachage" dans votre navigateur.

#### Build des images

In [ ]:
!git clone https://github.com/jpetazzo/container.training

In [ ]:
!docker build container.training/dockercoins/rng -t rng:v0.1 &>/dev/null
!docker build container.training/dockercoins/hasher -t hasher:v0.1 &>/dev/null
!docker build container.training/dockercoins/webui -t webui:v0.1 &>/dev/null
!docker build container.training/dockercoins/worker -t worker:v0.1 &>/dev/null

Ou bien on aurait aussi pu pull les images déjà buildées sur DockerHub
```shell
docker pull dockercoins/rng:v0.1
docker pull dockercoins/hasher:v0.1
docker pull dockercoins/webui:v0.1
docker pull dockercoins/worker:v0.1
```

#### Pull de l'image Redis officielle

In [ ]:
!docker pull redis:latest

#### On run chaque service

In [ ]:
!docker network create dockercoins

In [ ]:
!docker run -d -p 8001:80 --name=rng --network=dockercoins rng:v0.1
!docker run -d -p 8002:80 --name=hasher --network=dockercoins hasher:v0.1
!docker run -d -p 8000:80 --name=webui --network=dockercoins webui:v0.1
!docker run -d --name=worker --network=dockercoins worker:v0.1
!docker run -d -p 6379:6379 --name=redis --network=dockercoins redis:latest

http://localhost:8000/index.html

#### Clean des containers

Pour windows: ajouter "powershell" devant chaque commande
Ex: !powershell docker ps -a -q

In [ ]:
# to list active containers
!docker ps -a -q
# to stop all running containers
!docker stop $(docker ps -a -q)
# to kill all running containers
!docker kill $(docker ps -q)
# to removed stopped running containers
!like: docker rm $(docker ps -a -q) --force # like: docker container prune --force
!docker network rm dockercoins

## II. Déployer l'application Dockercoins avec un docker compose

In [ ]:
!cat container.training/dockercoins/compose.yml

Depuis container.training/dockercoins:

In [ ]:
!cd container.training/dockercoins ; docker compose up

C'est quand même plus simple non ?

http://localhost:8000/index.html

#### Clean des containers

In [ ]:
# to stop all running containers
!docker stop $(docker ps -a -q)
# to kill all running containers
!docker kill $(docker ps -q)
# to removed stopped running containers
!docker container prune --force